In [1]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings
import re

import config
from yandex_disk_func import *
from parse_functions import *

pd.options.mode.chained_assignment = None

# # указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\35_mediaplan_parser'
# file_name = 'yandex_token.txt'

# # сохраняем токен в отдельную переменную
yandex_token = config.yandex_token


# # указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = config.main_folder
public_key = config.public_key # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [3]:
# Создаем функцию для получения содержимого Яндекс диска
#  на входе принимаем 
# - адрес запроса для скачивания файла
# - public key, который мы получаем из ответа Яндекс при запросе информации по конкретной папке
# - адрес папки
# - лимит файлов (максимальное кол-во 80)
def get_yandex_disk_responce(request_url, public_key, folder_path, limit=80):
    res_url = request_url + urlencode(dict(public_key=public_key, path=folder_path, limit=limit)) # формируем строку запроса
    res = requests.get(res_url) # отправляем запрос на сервер
    yandex_responce = res.json() # получаем ответ и преобразуем его в json

    return yandex_responce

In [4]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(report_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])

/01_yandex
/02_prog
FirstData_Игроник_Национальная лотерея_Принцесса Нури_СРМ_media_27.08.2024 (1)
/03_Полина


In [5]:
data_link = download_response.content

In [6]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [7]:
sheet_names.sheet_names

['mediaplan 1000к ',
 'mediaplan 800к',
 'mediaplan 600к',
 'Brief_MEDIA',
 'Материалы для старта']

In [8]:
# функция для нормализации строк в датаФрейме
# на входе принимает поле со строковыми данными
# приводит в нижний регистр / обрезает пробелы / удаляет символ переноса строки
def normalize_text(column):
    return column.str.lower().str.strip().str.replace('\n', ' ')

In [9]:
# функция для нормализации заголовков в датаФрейме
# на входе принимает строка отдельного заголовка
# приводит в нижний регистр / обрезает пробелы / удаляет символ переноса строки / удаляем двойные пробелы
def normalize_headers(column):
    column = column.lower().strip().replace('\n', ' ')
    column = column.replace('*', '')
    column = re.sub(' +', ' ', column)

    return column

In [10]:
def normalize_digits(column):
    column = str(column)
    if '-' in column:
        column = '0'
    return column

In [11]:
# если в поле с числом содержатся буквы - например 100руб.
# функция оставит только числа
# если в поле нет букв, только число, то вернется число
def get_digits(column):
    try:
        res = re.sub('\D', '', column)
    except:
        res = column
    return res

In [12]:
network = 'FirstData'
tmp_dict = {}
for sheet_name in sheet_names.sheet_names:
    
    if 'mediaplan' in sheet_name:
        print(sheet_name)
        df = pd.read_excel(BytesIO(data_link), sheet_name=sheet_name)
        # заполняем вниз название истоника
        df['Unnamed: 1'] = df['Unnamed: 1'].ffill()
        # заполняем вниз таргетинги
        df['Unnamed: 2'] = df['Unnamed: 2'].ffill()
        # заполяем вниз rotation type - здесь объединенная ячейка, и в этом поле нет названия
        # чтобы оно появилось, заранее протягиваем вниз это название
        df['Unnamed: 8'] = df['Unnamed: 8'].fillna('rotation type')
        # заполняем вниз объединенные ячейки
        df = df.fillna('')
        # забираем индекс строки, где находится название Бренда
        brand_index = list(df[df['Unnamed: 2'].str.lower().str.contains('brand')].index)[0] 
        # сохраняем название бренда
        brand = df['Unnamed: 3'].loc[brand_index] 
        # забираем период медиаплана
        period = df['Unnamed: 3'].loc[brand_index+2] 
        # забираем индекс начала таблицы
        start_index = list(df[df['Unnamed: 1'].str.lower().str.contains('category')].index)[0]
        # задаем названия полей
        df.columns = df.iloc[start_index].apply(normalize_headers) # забираем название полей из файла
        # обрезаем верхнюю часть таблицы. она больше не нужна
        df = df.iloc[start_index+2:].reset_index(drop=True)
        # забираем окончание таблицы
        end_index =list(df[df['category'].str.lower().str.contains('total')].index)[0]
        # обрезаем таблицу снизу
        df = df.iloc[:end_index].reset_index(drop=True)
        # создаем базовый список полей, которые есть всегда вне зависимости от типа размещения
        base_columns = ['category', 'targeting by purchase', 'format', 'quantity of units', 'period', 
                        'price list cost (cost per unit) net ', 'rotation type',
                        'total price list cost net', 'reach forecast (uu)', 'frequency total till',
                        'impressions', 'clicks']
        
        # проверяем наличие Видео размещений. Если они есть, то используем дополнительные поля из таблицы
        # создаем список уникальных форматов и преобразуем его в одну строку
        format_string = ' '.join(list(df['format'].drop_duplicates().apply(lambda x: normalize_headers(x))))

        if 'видео' in format_string:
            base_columns += ['number of views', 'vtr,%']

        df = df[base_columns]
        df = df.rename(columns={'category': 'source', 'targeting by purchase': 'targeting', 'format': 'ad copy format', 
                                'quantity of units': 'unit quantity', 'price list cost (cost per unit) net ': 'unit price', 
                                'frequency total till': 'frequency', 'reach forecast (uu)': 'reach', 
                                'total price list cost net' :'budget_without_nds'})
        
        if 'видео' in format_string:
            df = df.rename(columns={'number of views': 'views', 'vtr,%': 'vtr, %'})
        # некоторые типы размещений имеют объединенные строки
        # например Баннер и универсальный баннер - это 2 строки с объединенными ячейками по расходам, показам и тд.
        # поэтому создадим доп. поле, где соединим их названия в одну строку
        # создаем пустое поле
        df['merge_type_cells'] = ''
        # проходим через цикл по датаФрейму
        # в первой строке по определению не может быть данных, поэтому сохраняем там название формата
        # если это не первая и не посленяя строка, то нам нужно провести проверку
        # допустим мы находимся в строке номер 2
        # мы проверяем, что находится в строке номер 3 в поле rotation type (на первых шагах мы сделали заполнение вниз)
        # соответсвенно если ячейка была пустая, там появится надпись rotation type - так мы поймем, что это как раз объединенные данные
        # берем название формата из текущей строки и добавляем к нему название формата из следующей строки
        # во всех остальных случаях просто возвращаем название формата из текущей строки
        for i in range(len(df)):
            base_name = df['ad copy format'][i]
        
            if i < len(df)-1:
                if df['rotation type'][i+1] == 'rotation type':
                    base_name = base_name + ' / ' + str(df['ad copy format'][i+1])
            else:
                base_name = df['ad copy format'][i]
            df['merge_type_cells'][i] =  base_name

        # забираем окончание таблицы
        end_index = list(df[df['unit quantity']==''].index)[0]
        # обрезаем таблицу снизу
        df = df.iloc[:end_index].reset_index(drop=True)
        # передаем новое название формата в нужное нам поле
        df['ad copy format'] = df['merge_type_cells']
        df = df.drop('merge_type_cells', axis=1)
        

        df['supplier'] = network
        df['report_name'] = sheet_name
        df['brand'] = brand
        df['period'] = period
        df['site/ssp'] = ''
        df['placement'] = ''
        df['budget_nds'] =(df['budget_without_nds'] * 1.2).astype('float').round(2)
        
        # нормализуем текстовые поля
        text_columns = ['report_name', 'brand', 'period', 'source', 'site/ssp', 'placement', 'targeting', 'ad copy format', 'rotation type']
        df[text_columns] = df[text_columns].apply(normalize_text)
        
        # нормализуем целые числа
        int_columns = ['unit quantity', 'reach', 'impressions', 'clicks']
        df[int_columns] = df[int_columns].apply(lambda x: x.astype('int'))
        # к десятичным приводим только некоторые колонки, т.к. в остальных может встречаться пусто - оно не конвертируется в число
        # не хочется тратить на это время
        df['budget_without_nds'] = df['budget_without_nds'].astype('float')


        base_cols = ['supplier', 'report_name', 'brand', 'period', 'source', 'site/ssp', 'placement', 'targeting', 'ad copy format', 'rotation type',\
                            'unit quantity', 'unit price', 'frequency', 'reach', 'impressions', 'clicks', 'budget_without_nds', 'budget_nds']
        if 'views' not in list(df.columns):
            df['views'] = 0
            df['vtr, %'] = 0.0
            media_cols = ['views', 'vtr, %']
            base_cols += media_cols

        df = df[base_cols]
        tmp_dict[sheet_name] = df

    # return pd.concat(tmp_dict, ignore_index=True)

mediaplan 1000к 
mediaplan 800к
mediaplan 600к


In [13]:
tmp_df = pd.concat(tmp_dict, ignore_index=True)

In [14]:
tmp_df

7,supplier,report_name,brand,period,source,site/ssp,placement,targeting,ad copy format,rotation type,unit quantity,unit price,frequency,reach,impressions,clicks,budget_without_nds,budget_nds,views,"vtr, %"
0,FirstData,mediaplan 1000к,национальная лотерея // принцесса нури,6 months,платформа first data,,,"гео: рф ца: ж, 35-45. доход в, вс. покупатели чая и чайной продукции: принцесса нури, азерчай, ява, riston, майский, ахмад, dilmah, ява, милфорд, липтон и др. покупатели кондитерских изделий: безе, варенье, джем, повидло, грильяж, желе, зефир, пастила, конфеты, ирис, халва, помадка, муссы, шоколад, торты и др источники: офд",баннеры / универсальные баннеры,cpm,25000000,240,12,2083333,25000000,50000,6000000.0,7200000.0,0,0.0
1,FirstData,mediaplan 800к,национальная лотерея // принцесса нури,6 months,платформа first data,,,"гео: рф ца: ж, 35-45. доход в, вс. покупатели чая и чайной продукции: принцесса нури, азерчай, ява, riston, майский, ахмад, dilmah, ява, милфорд, липтон и др. покупатели кондитерских изделий: безе, варенье, джем, повидло, грильяж, желе, зефир, пастила, конфеты, ирис, халва, помадка, муссы, шоколад, торты и др источники: офд",баннеры / универсальные баннеры,cpm,20000000,240,12,1666666,20000000,40000,4800000.0,5760000.0,0,0.0
2,FirstData,mediaplan 600к,национальная лотерея // принцесса нури,6 months,платформа first data,,,"гео: рф ца: ж, 35-45. доход в, вс. покупатели чая и чайной продукции: принцесса нури, азерчай, ява, riston, майский, ахмад, dilmah, ява, милфорд, липтон и др. покупатели кондитерских изделий: безе, варенье, джем, повидло, грильяж, желе, зефир, пастила, конфеты, ирис, халва, помадка, муссы, шоколад, торты и др источники: офд",баннеры / универсальные баннеры,cpm,13900000,260,12,1158333,13900000,27800,3614000.0,4336800.0,0,0.0


In [ ]:

list(df[df['unit quantity']==''].index)[0]

In [ ]:
df['ad copy format'][0]

In [ ]:
df.iloc[:2]

In [ ]:
df.columns

In [ ]:
df = pd.read_excel(BytesIO(data_link), sheet_name='Plan_Media')
# заполняем вниз объединенные ячейки
df['Unnamed: 1'] = df['Unnamed: 1'].ffill()
df['Unnamed: 2'] = df['Unnamed: 2'].ffill()
df['Unnamed: 3'] = df['Unnamed: 3'].ffill()
df = df.fillna('')
# забираем индекс строки, где находится название Бренда
brand_index = list(df[df['Unnamed: 1'].str.lower().str.contains('brand')].index)[0] 
# сохраняем название бренда
brand = df['Unnamed: 2'].loc[brand_index] 
# забираем период медиаплана
period = df['Unnamed: 2'].loc[brand_index+1] 
# забираем индекс начала таблицы
start_index = list(df[df['Unnamed: 1'].str.lower().str.contains('source')].index)[0]
# задаем названия полей
df.columns = df.iloc[start_index].apply(normalize_headers) # забираем название полей из файла
# обрезаем верхнюю часть таблицы. она больше не нужна
df = df.iloc[start_index+2:].reset_index(drop=True)
# забираем окончание таблицы
end_index = list(df[df['source'].str.lower().str.contains('итого')].index)[0]
# обрезаем таблицу снизу
df = df.iloc[:end_index].reset_index(drop=True)

# # # определяем к какрму типу рекламы относятся строки
if 'banners' in list(df['ad copy format'].drop_duplicates().apply(normalize_headers)) \
and len(list(df['ad copy format'].drop_duplicates().apply(normalize_headers)))<2:
    # оставляем только нужные поля
    df = df[['source', 'site/ssp', 'placement', 'targeting', 'ad copy format', 'unit', 'rotation type', 
     'unit quantity', 'unit price', 'frequency', 'reach', 'impressions', 'clicks', 'ratecard price per period (rub, net)']]
else:
    df = df[['source', 'site/ssp', 'placement', 'targeting', 'ad copy format', 'unit', 'rotation type', 
     'unit quantity', 'unit price', 'frequency', 'reach', 'impressions', 'clicks', 'views', 'vtr, %', 'ratecard price per period (rub, net)']]
    # убираем знак -, если он есть
    # убираем знак рубля, если он есть в стоимости

    
    df['views'] = df['views'].apply(normalize_digits)
    df['vtr, %'] = df['vtr, %'].apply(normalize_digits)
    # нормализуем десятичные числа
    df['views'] = df['views'].astype('int')
    df['vtr, %'] = df['vtr, %'].astype('float')

 # создаем список с названиями текстовых полей для их нормализации
df = df.rename(columns={'ratecard price per period (rub, net)': 'budget_without_nds'})
df['brand'] = brand
df['period'] = period
text_columns = ['brand', 'period', 'source', 'site/ssp', 'placement', 'targeting', 'ad copy format', 'rotation type']
df[text_columns] = df[text_columns].apply(normalize_text)

# убираем знак рубля, если он есть в стоимости
currecny_columns = ['unit', 'unit quantity']
df[currecny_columns] = df[currecny_columns].apply(get_digits)

# нормализуем целые числа
int_columns = ['unit quantity', 'reach', 'impressions', 'clicks']
df[int_columns] = df[int_columns].apply(lambda x: x.astype('int'))

# добавляем рассчитываемые показатели
# df['budget_without_nds'] = ((df['unit quantity'] / 1000) * df['unit price']).astype('float')
df['budget_without_nds'] = df['budget_without_nds'].astype('float')
df['budget_nds'] =(df['budget_without_nds'] * 1.2).astype('float').round(2)


In [ ]:
df

In [ ]:


# добавляем рассчитываемые показатели
df['budget'] = (df['unit quantity'] / 1000) * df['unit price']
df['ctr%'] = ((df['clicks'] / df['impressions'])*100).round(2)
df['cpc'] = (df['budget'].astype('float') / df['clicks'].astype('float')).round(2)
df['cpm'] = ((df['budget'].astype('float') / df['impressions'].astype('float'))*1000).round(2)

In [ ]:
list(df['ad copy format'].drop_duplicates().apply(normalize_headers, axis=1))

In [ ]:
df.head()

In [ ]:
ad_type

In [ ]:
df.columns

In [ ]:
df['ad copy  format']

In [ ]:
df

In [ ]:

df = pd.read_excel(BytesIO(data_link), sheet_name='Plan_Media')
# заполняем вниз объединенные ячейки
df['Unnamed: 1'] = df['Unnamed: 1'].ffill()
df['Unnamed: 2'] = df['Unnamed: 2'].ffill()
df['Unnamed: 3'] = df['Unnamed: 3'].ffill()
df = df.fillna('')
# забираем индекс строки, где находится название Бренда
brand_index = list(df[df['Unnamed: 1'].str.lower().str.contains('brand')].index)[0] 
# сохраняем название бренда
brand = df['Unnamed: 2'].loc[brand_index] 
# забираем период медиаплана
period = df['Unnamed: 2'].loc[brand_index+1] 
# забираем индекс начала таблицы
start_index = list(df[df['Unnamed: 1'].str.lower().str.contains('source')].index)[0]
# задаем названия полей
df.columns = df.iloc[start_index].str.lower().str.strip().str.replace('\n', ' ') # забираем название полей из файла
# обрезаем верхнюю часть таблицы. она больше не нужна
df = df.iloc[start_index+2:].reset_index(drop=True)
# забираем окончание таблицы
end_index = list(df[df['source'].str.lower().str.contains('итого')].index)[0]
# обрезаем таблицу снизу
df = df.iloc[:end_index].reset_index(drop=True)

# определяем к какрму типу рекламы относятся строки
ad_type = df['ad copy format'].loc[0]

if 'banner' in ad_type.lower():
    # оставляем только нужные поля
    df = df[['source', 'site/ssp', 'placement', 'targeting', 'ad copy format', 'unit', 'rotation type', 
     'unit quantity', 'unit price', 'frequency', 'reach', 'impressions', 'clicks']]
# создаем список с названиями текстовых полей для их нормализации
text_columns = ['source', 'site/ssp', 'placement', 'targeting', 'ad copy format', 'rotation type']
df[text_columns] = df[text_columns].apply(normalize_text)

# убираем знак рубля, если он есть в стоимости
currecny_columns = ['unit', 'unit quantity']
df[currecny_columns] = df[currecny_columns].apply(get_digits)





In [ ]:
df = pd.read_excel(BytesIO(data_link), sheet_name='Statistics by creative', skiprows=1)


In [ ]:
len(df)